In [1]:
import psycopg2

In [4]:
from teams_notifications import channel_notifications

def rds_connection(username, password):

    """
    Will attempt to connect to the AWS RDS Database using provided username and password.  Host and database are
    already set for the Tidal Basin environment. If successful, the connection will be returned, if the connection fails
    a teams channel will be notified and a raise exception will be passed stopping the python scripting process.
    
    """

    try:

        #Connect to the AWS RDS Database
        conn = psycopg2.connect(
            host = 'tidalbasin-cdbg-dr-fl-ida-backendauroraclusterclus-sefctthwsqmq.ce8xd2bu2edx.us-east-2.rds.amazonaws.com',
            port = 5432,
            user = username,
            password = password,
            database= 'tidalbasin'
        )

        #Return the Connection Object if Successful
        return conn
    

    except Exception as e:

        #Notify Teams Channel that Connection Failed to AWS RDS
        channel_notifications(alert_type = "Error", 
                              summary = "RDS Connection Failure", 
                              message = "Failed to Connect to the AWS RDS Database",
                              error = e)
        

        # Raise Exception to Stop Process if Failure
        raise Exception(f"Failed to Connect to RDS Database: {e}")

OperationalError: connection to server at "tidalbasin-cdbg-dr-fl-ida-backendauroraclusterclus-sefctthwsqmq.ce8xd2bu2edx.us-east-2.rds.amazonaws.com" (10.0.136.135), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
cur = conn.cursor()

# Execute a SQL query to pull data from a table
query = "SELECT * FROM YOUR_TABLE_NAME"
cur.execute(query)

# Fetch the data and print it
rows = cur.fetchall()
for row in rows:
    print(row)

# Close the cursor and connection
cur.close()
conn.close()